In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

import settings as conf
from entity import Trait

# Load OMIM silver standard

In [3]:
omim_silver_standard = pd.read_csv(conf.OMIM_SILVER_STANDARD_FINAL_FILE, sep='\t')

In [4]:
omim_silver_standard = omim_silver_standard.dropna(subset=['ensembl_gene_id', 'trait', 'pheno_mim'])

In [5]:
display(omim_silver_standard.shape)
display(omim_silver_standard.head())

(7165, 7)

,trait,pheno_mim,mim,entry_type,entrez_gene_id,approved_gene_symbol,ensembl_gene_id
0,22130-Doctor_diagnosed_COPD_chronic_obstructiv...,103580,139320,gene,2778,GNAS,ENSG00000087460
1,22130-Doctor_diagnosed_COPD_chronic_obstructiv...,105830,601623,gene,7337,UBE3A,ENSG00000114062
2,22130-Doctor_diagnosed_COPD_chronic_obstructiv...,139210,600993,gene,4089,SMAD4,ENSG00000141646
3,22130-Doctor_diagnosed_COPD_chronic_obstructiv...,170100,613230,gene,5184,PEPD,ENSG00000124299
4,22130-Doctor_diagnosed_COPD_chronic_obstructiv...,176270,602117,gene,4692,NDN,ENSG00000182636


# Generate list of traits

In [6]:
ukb_traits_mapped = pd.Series(omim_silver_standard['trait'].unique())
display(ukb_traits_mapped.head())

0    22130-Doctor_diagnosed_COPD_chronic_obstructiv...
1    I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic...
2    M81-Diagnoses_main_ICD10_M81_Osteoporosis_with...
3                          1200-Sleeplessness_insomnia
4    20002_1459-Noncancer_illness_code_selfreported...
dtype: object

In [7]:
ukb_traits_mapped.shape

(104,)

In [8]:
ukb_codes_mapped = [Trait(full_code=t).code for t in ukb_traits_mapped]

In [9]:
len(ukb_codes_mapped)

104

In [10]:
ukb_codes_mapped[:5]

['22130', 'I25', 'M81', '1200', '20002_1459']

In [11]:
# This is needed to run gwas2gene
pd.Series(ukb_codes_mapped).to_csv('../extras/gwas2gene/selected_ukb_traits_omim.txt', index=False, header=False)

# Run gwas2gene

In [12]:
n_jobs = conf.N_JOBS
display(n_jobs)

2

In [13]:
gwas_parsing_dir = conf.GWAS_PARSING_FULL_DIR
display(gwas_parsing_dir)

'/home/miltondp/projects/labs/hakyimlab/phenomexcan/base/results/misc/gwas_parsing/full/'

In [14]:
gwas2gene_results_dir = conf.OMIM_SILVER_STANDARD_GWAS_TO_GENE_DIR
display(gwas2gene_results_dir)

'/home/miltondp/projects/labs/hakyimlab/phenomexcan/base/results/omim_silver_standard/gwas2gene'

In [15]:
%%bash -s "$gwas_parsing_dir" "$n_jobs" "$gwas2gene_results_dir"

cd ../extras/gwas2gene/
cat selected_ukb_traits_omim.txt | parallel -j${2} "bash run_gwas2gene.sh ${1}/{}.txt.gz"
mkdir -p ${3}
mv _results/* ${3}


Attaching package: ‘dplyr’

The following objects are masked from ‘package:data.table’:

    between, first, last

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Taking input= as a system command ('zcat /home/miltondp/projects/labs/hakyimlab/phenomexcan/base/results/misc/gwas_parsing/full//22130.txt.gz | cut -f 2,3,4,8,11') and a variable has been used in the expression passed to `input=`. Please use fread(cmd=...). There is a security concern if you are creating an app, and the app could have a malicious user, and the app is not running in a secure envionment; e.g. the app is running as root. Please read item 5 in the NEWS file for v1.11.6 for more information and for the option to suppress this message.

Attaching package: ‘dplyr’

The following objects are masked from ‘package:data.table’:

    between, first, last

The following objects are masked from ‘package